# ALeRCE Xmatch service example

A short example for accessing the courrent data aviable on the ALeRCE servers for cross-match, using custom arbirtrary objects

In [1]:
!pip install alerce_xmatch

  Using cached https://files.pythonhosted.org/packages/96/4e/cbde3cf4b59748d4d465db8f8f769283f2a580d2abb3ba8b889fd52efe47/alerce_xmatch-0.0.2-py3-none-any.whl


For documentation visit GitHub repository: https://github.com/alercebroker/xmatch_client

Or the PyPi project: https://pypi.org/project/alerce-xmatch/

Generate data example

In [2]:
import pandas as pd

In [3]:
#input
columns = ['oid', 'ra', 'dec']
values = [
    [0, 168.821096816667, 29.6921437083333],
    [1, 164.669160169091, 16.0180675727273],
    [2, 174.65028162381, 20.726187647619],
    [3, 154.4155987, 30.8199994785714]
]
df = pd.DataFrame(values, columns=columns)

## Using Catalog class

In [4]:
from alerce_xmatch import Catalog, TargetCatalog, OutputCols
import time

Create Catalog class with test data

In [5]:
my_catalog = Catalog(df=df, name="My catalog")

## Crossmatch with Gaia (a Catalog in DataBase)

start = time.time()

cross_catalog = my_catalog.crossmatch(TargetCatalog.GAIA)

end = time.time()

print("Catalog name: {}".format(cross_catalog.name))
print("Crossmatch take: {} s".format(end - start))
print("Result:")
cross_catalog.df

Catalog name: My catalog
Crossmatch take: 2.415166139602661 s
Result:


,My catalog_dec,My catalog_oid,My catalog_ra,GAIA-DR2_designation,GAIA-DR2_ra,GAIA-DR2_dec,GAIA-DR2_parallax,GAIA-DR2_pmra,GAIA-DR2_pmdec,GAIA-DR2_astrometric_pseudo_colour,...,GAIA-DR2_bp_rp,GAIA-DR2_bp_g,GAIA-DR2_g_rp,GAIA-DR2_radial_velocity,GAIA-DR2_teff_val,GAIA-DR2_lum_val,GAIA-DR2_classifier_name,GAIA-DR2_best_class_name,GAIA-DR2_best_class_score,separation
0,16.018068,1.0,164.66916,Gaia DR2 3981758871818873600,164.669179,16.018048,0.107938,-3.091704,-2.052673,1.652324,...,0.572648,0.185996,0.386652,NaN,6898.6665,NaN,nTransits:2+,RRAB,0.985119,0.093911


To much info? Let's do a crossmatch with a catalog in memory (it supposed to be faster) and just see our info

In [6]:
start = time.time()

cross_catalog = my_catalog.crossmatch(TargetCatalog.ASASSN, output=OutputCols.SOURCE)  # Just my info please

end = time.time()

print("Crossmatch take: {} s".format(end - start))
print("Result:")
cross_catalog.df

Crossmatch take: 1.099226951599121 s
Result:


,separation,My catalog_oid,My catalog_ra,My catalog_dec
0,None,0.0,168.821097,29.692144
1,None,3.0,154.415599,30.819999
2,None,1.0,164.669160,16.018068
3,None,2.0,174.650282,20.726188


I know the ra, dec position, maybe that's info is not neccesary

In [7]:
cross_catalog = my_catalog.crossmatch(TargetCatalog.ZTF, output=OutputCols.TARGET, # Just yuor info
                                     radec=False) # Don't show me the position
cross_catalog.df

,separation,ZTF_index,ZTF_oid
0,None,2186554,ZTF19aadfkyz
1,None,7331389,ZTF19aadfkza
2,None,10821724,ZTF19aaknuwq
3,None,14538933,ZTF18aabksxe
4,None,14630424,ZTF18aaanyxu
5,None,14731266,ZTF18aaxpvha
6,None,14738882,ZTF17aabtute
7,None,14738883,ZTF17aaajmde
8,None,15137915,ZTF18acbwbhp


## Using just the DataFrame
If you don't want to use that annoying Catalog class, use the crossmatch static method instead

In [8]:
from alerce_xmatch import crossmatch

In [9]:
crossmatch(df, TargetCatalog.ZTF) # df was declared on cell 3

,ZTF_index,ZTF_oid,ZTF_ra,ZTF_dec,ra,dec,separation,input_catalog_oid,input_catalog_ra,input_catalog_dec
0,2186554,ZTF19aadfkyz,154.415950,30.819799,154.415774,30.819899,None,3.0,154.415599,30.819999
1,7331389,ZTF19aadfkza,154.416161,30.819826,154.415880,30.819913,None,3.0,154.415599,30.819999
2,10821724,ZTF19aaknuwq,164.669156,16.017554,164.669158,16.017811,None,1.0,164.669160,16.018068
3,14538933,ZTF18aabksxe,168.821101,29.692145,168.821099,29.692144,None,0.0,168.821097,29.692144
4,14630424,ZTF18aaanyxu,168.821001,29.692204,168.821049,29.692174,None,0.0,168.821097,29.692144
5,14731266,ZTF18aaxpvha,164.669518,16.018026,164.669339,16.018047,None,1.0,164.669160,16.018068
6,14738882,ZTF17aabtute,164.669162,16.018068,164.669161,16.018068,None,1.0,164.669160,16.018068
7,14738883,ZTF17aaajmde,174.650281,20.726185,174.650281,20.726186,None,2.0,174.650282,20.726188
8,15137915,ZTF18acbwbhp,154.415609,30.819991,154.415604,30.819995,None,3.0,154.415599,30.819999


You can also use the crossmatch method with a Catalog object, and the output will be a Catalog as well

In [10]:
crossmatch(my_catalog, TargetCatalog.LINEAR).df # Remember print just the DataFrame

,LINEAR_id,LINEAR_class,LINEAR_period,LINEAR_A,LINEAR_mmed,LINEAR_stdev,LINEAR_rms,LINEAR_Lchi2pdf,LINEAR_nObs,LINEAR_skew,...,LINEAR_gErr,LINEAR_rErr,LINEAR_iErr,LINEAR_zErr,ra,dec,separation,My catalog_oid,My catalog_ra,My catalog_dec
0,1863652,RR Lyr ab,0.635622,0.88,16.68,0.31,0.38,1.671,250,-0.29,...,0.01,0.01,0.01,0.01,174.650267,20.726207,None,2.0,174.650282,20.726188
1,23555046,RR Lyr ab,0.514367,0.77,15.62,0.25,0.27,2.268,303,-0.66,...,0.01,0.01,0.01,0.01,164.669183,16.018061,None,1.0,164.669160,16.018068


## An example with a bigger catalog

To check a bigger crossmatch, we're gonna use the _catalina.pkl_ dataframe

In [11]:
import pickle

with open("example_data/catalina.pkl", "rb") as file:
    catalina = pickle.load(file)
    
catalina

,oid,ra,dec
0,CSS_J000020.4+103118,0.085042,10.521917
1,CSS_J000031.5-084652,0.131250,-8.781194
2,CSS_J000036.9+412805,0.153917,41.468250
3,CSS_J000037.5+390308,0.156458,39.052250
4,CSS_J000103.3+105724,0.264042,10.956778
5,CSS_J000103.4+395744,0.264417,39.962361
6,CSS_J000106.9+120610,0.279000,12.102861
7,CSS_J000110.8+400521,0.295375,40.089194
8,CSS_J000131.5+324913,0.381417,32.820306
9,CSS_J000141.2+421108,0.422000,42.185611


In [14]:
# Create catalog:
catalina_catalog = Catalog(df=catalina, name="Catalina Example")

catalina_catalog.crossmatch(TargetCatalog.ASASSN).df

,ASASSN_id,ASASSN_Other Names,ASASSN_LCID,ASASSN_ra,ASASSN_dec,ASASSN_Mean VMag,ASASSN_Amplitude,ASASSN_period,ASASSN_class,ASASSN_Url,...,ASASSN_Sllk Statistic,ASASSN_RF Regression Score,ASASSN_Classification Probability,ASASSN_Epoch (HJD),ra,dec,separation,Catalina Example_oid,Catalina Example_ra,Catalina Example_dec
0,ASASSN-V J204308.90-011533.7,CSS_J204308.9-011532,175106,310.78710,-1.25937,15.16,0.38,0.4294613,EB,/database/light_curves/283889,...,0.28,0.86,0.998,2.457904e+06,310.787175,-1.259241,None,CSS_J204308.9-011532,310.787250,-1.259111
1,ASASSN-V J013051.79+313851.1,AM Psc,51150,22.71578,31.64752,13.83,0.80,107.8898995,SR,/database/light_curves/381090,...,0.60,0.70,1.000,2.457588e+06,22.715744,31.647552,None,CSS_J013051.7+313851,22.715708,31.647583
2,ASASSN-V J234254.97+384328.2,CSS_J234255.0+384328,131254,355.72903,38.72450,14.83,0.93,216.351107,SR,/database/light_curves/382465,...,0.73,0.66,1.000,2.457309e+06,355.729223,38.724486,None,CSS_J234255.0+384328,355.729417,38.724472
3,ASASSN-V J165903.31+433053.6,V0782 Her,56887,254.76379,43.51490,14.19,1.70,228.2831729,SR,/database/light_curves/382544,...,0.26,0.87,1.000,2.458019e+06,254.763812,43.514881,None,CSS_J165903.3+433053,254.763833,43.514861
4,ASASSN-V J055246.35-024207.0,CSS_J055246.3-024207,424610,88.19312,-2.70194,15.55,0.46,2.1161883,EA,/database/light_curves/286677,...,0.32,0.84,0.998,2.457335e+06,88.193122,-2.701942,None,CSS_J055246.3-024207,88.193125,-2.701944
5,ASASSN-V J102157.76-034342.0,BW Sex,332925,155.49068,-3.72833,11.47,1.12,1.5378266,EA,/database/light_curves/288185,...,0.01,0.99,0.998,2.457917e+06,155.490757,-3.728262,None,CSS_J102157.8-034341,155.490833,-3.728194
6,ASASSN-V J020110.75+334715.9,RY Tri,83008,30.29478,33.78774,13.98,1.12,204.2456981,SR,/database/light_curves/385150,...,0.25,0.87,1.000,2.456518e+06,30.294932,33.787495,None,CSS_J020110.8+334714,30.295083,33.787250
7,ASASSN-V J205223.45-162605.1,NSVS J2052237-162609,425951,313.09772,-16.43474,14.52,0.98,185.2263885,SR,/database/light_curves/385896,...,0.61,0.71,1.000,2.456752e+06,313.097881,-16.434801,None,CSS_J205223.5-162605,313.098042,-16.434861
8,ASASSN-V J160402.12+084604.8,LINEAR 16384002,73900,241.00882,8.76801,16.20,0.98,132.9234414,SR,/database/light_curves/385952,...,0.27,0.87,1.000,2.456833e+06,241.008972,8.768130,None,CSS_J160402.1+084605,241.009125,8.768250
9,ASASSN-V J043834.91+202621.6,CSS_J043834.9+202621,144627,69.64544,20.43934,16.18,1.44,159.8899521,SR,/database/light_curves/386100,...,0.25,0.88,1.000,2.457037e+06,69.645470,20.439267,None,CSS_J043834.9+202621,69.645500,20.439194


In [16]:
# Crossmatch with ZTF
catalina_catalog.crossmatch(TargetCatalog.ZTF).df

,ZTF_index,ZTF_oid,ZTF_ra,ZTF_dec,ra,dec,separation,Catalina Example_oid,Catalina Example_ra,Catalina Example_dec
0,431,ZTF18abafuwt,161.972995,41.240032,161.972977,41.239891,None,CSS_J104753.5+411423,161.972958,41.239750
1,874,ZTF18abcrjek,349.174501,36.557156,349.174521,36.557050,None,CSS_J231641.8+363325,349.174542,36.556944
2,2967,ZTF18aborcee,341.858546,26.772723,341.858294,26.772723,None,CSS_J224725.9+264621,341.858042,26.772722
3,3189,ZTF18abqlpok,339.243784,23.899569,339.243746,23.899590,None,CSS_J223658.4+235358,339.243708,23.899611
4,6990,ZTF18achbvgn,80.723006,-2.473349,80.723024,-2.473286,None,CSS_J052253.5-022823,80.723042,-2.473222
5,12018,ZTF19aaeqkti,128.303213,-1.534178,128.303398,-1.534186,None,CSS_J083312.8-013203,128.303583,-1.534194
6,14504,ZTF19aatjgmf,152.003311,-4.411925,152.003301,-4.412157,None,CSS_J100800.7-042444,152.003292,-4.412389
7,15037,ZTF17aackhvb,197.124540,51.111280,197.124707,51.111320,None,CSS_J130829.9+510640,197.124875,51.111361
8,15281,ZTF18aabplzw,164.789330,40.824968,164.789186,40.824984,None,CSS_J105909.3+404930,164.789042,40.825000
9,15737,ZTF18aakgzow,247.677162,53.216244,247.677144,53.216275,None,CSS_J163042.5+531258,247.677125,53.216306
